In [1]:
from pyconnect.spark import SparkCluster
sparkcluster = SparkCluster().getOrCreate(
  'spark-cluster-pyconnect', 
  minExecutors=2,
  executorMemory = "4g",
  driverMemory = "4g" )

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import trim


from pyspark.sql import SQLContext

import numpy as np
import pandas as pd
from pandas import DataFrame as df
import difflib
from datetime import datetime as dte
import random
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns



hc = sparkcluster.session

SparkSession is created.
Check http://xhadrevrm2p.aetna.com:10001 for spark job UI

The application ID for the spark job is application_1621777416540_17086
If you exist without stopping the spark session, use the following command to kill the job:
yarn application -kill application_1621777416540_17086


In [5]:
df = pd.merge(df,wbc[['proxy_id', 'call_type']] , how  = 'left')

In [6]:
df.proxy_id.count()

120207

In [7]:
df = df [df['decile'] <= 5 ]
df.proxy_id.count()

120134

In [8]:
df.groupby(['decile_group']).proxy_id.count()

decile_group
Bottom 45    59801
Mid23        31614
Top1         10943
Name: proxy_id, dtype: int64

In [9]:
df.groupby(['expe_group']).proxy_id.count()

expe_group
HH     4017
HT    50065
TH     5025
TT    61027
Name: proxy_id, dtype: int64

In [10]:
import statsmodels.api as sm

In [11]:
X = df[['decile']]

In [12]:
catg_col = ['decile']

In [13]:
X = pd.get_dummies(X, columns = catg_col, prefix = catg_col, drop_first = True)

In [14]:
Y = df['expe_group']

In [15]:
X_1=sm.add_constant(X)

In [16]:
Y.head()

0    HT
1    HT
2    HT
3    HT
4    HT
Name: expe_group, dtype: object

In [17]:
mnlogit_model=sm.MNLogit(Y,X_1)
result1=mnlogit_model.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.952689
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:             expe_group   No. Observations:               120134
Model:                        MNLogit   Df Residuals:                   120116
Method:                           MLE   Df Model:                           15
Date:                Mon, 24 May 2021   Pseudo R-squ.:                0.002651
Time:                        06:08:03   Log-Likelihood:            -1.1445e+05
converged:                       True   LL-Null:                   -1.1475e+05
Covariance Type:            nonrobust   LLR p-value:                5.660e-120
expe_group=HT       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.2336      0.607      3.677      0.000       1.043       3.424
decile_1          0.

In [18]:
result1.predict(X_1)

,0,1,2,3
0,0.032898,0.448825,0.038218,0.480059
1,0.036019,0.444071,0.038429,0.481481
2,0.032898,0.448825,0.038218,0.480059
3,0.032978,0.388598,0.048713,0.529710
4,0.036019,0.444071,0.038429,0.481481
...,...,...,...,...
120202,0.036525,0.406253,0.044414,0.512808
120203,0.036019,0.444071,0.038429,0.481481
120204,0.032898,0.448825,0.038218,0.480059
120205,0.036525,0.406253,0.044414,0.512808


In [19]:

a = pd.DataFrame(result1.predict(X_1))

In [20]:
a.rename(columns = {0:'HH',1: 'H_WBC_T_MSV', 2:'T_WBC_H_MSV', 3 : 'TT'}, inplace= True)

In [21]:
a

,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT
0,0.032898,0.448825,0.038218,0.480059
1,0.036019,0.444071,0.038429,0.481481
2,0.032898,0.448825,0.038218,0.480059
3,0.032978,0.388598,0.048713,0.529710
4,0.036019,0.444071,0.038429,0.481481
...,...,...,...,...
120202,0.036525,0.406253,0.044414,0.512808
120203,0.036019,0.444071,0.038429,0.481481
120204,0.032898,0.448825,0.038218,0.480059
120205,0.036525,0.406253,0.044414,0.512808


In [22]:
df

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,return_pilot_t,return_pilot_h,decile,lis_ind,vol_churn,sales_channel,product_segment,expe_group,decile_group,call_type
0,001LGBBBBPXY,1,1,0,0,0,1,0,0,1,...,0,0,5,0,0,Strategic,HMO,HT,Bottom 45,Holdout
1,002317BBBPXY,1,1,0,0,0,1,0,0,1,...,0,0,4,0,0,NDP,HMO,HT,Bottom 45,Holdout
2,003RZGBBBPXY,1,1,0,0,0,1,0,0,1,...,0,0,5,0,0,remaining,PPO,HT,Bottom 45,Holdout
3,005RGQBBBPXZ,1,1,0,0,0,1,0,0,1,...,0,0,2,1,0,NDP,HMO,HT,None,None
4,00BGXGBBBPXY,1,1,0,0,0,1,0,0,1,...,0,0,4,0,0,remaining,PPO,HT,Bottom 45,Holdout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120202,ZZBXXKBBBPXY,1,1,0,0,1,0,0,0,1,...,0,0,3,0,0,NDP,PPO,TT,Mid23,Live
120203,ZZL2QQBBBPXZ,1,1,0,0,1,0,0,0,1,...,0,0,4,0,0,remaining,HMO,TT,Bottom 45,IVR
120204,ZZMXNKBBBPXY,1,1,0,0,0,1,0,0,1,...,0,0,5,0,0,NDP,PPO,HT,Bottom 45,Holdout
120205,ZZT2QQBBBPXZ,1,1,0,0,0,1,0,0,0,...,0,0,3,0,0,NDP,PPO,HH,Mid23,Holdout


In [23]:
data = pd.concat([df,a], axis = 1 )

In [24]:
data.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,vol_churn,sales_channel,product_segment,expe_group,decile_group,call_type,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT
0,001LGBBBBPXY,1,1,0,0,0,1,0,0,1,...,0,Strategic,HMO,HT,Bottom 45,Holdout,0.032898,0.448825,0.038218,0.480059
1,002317BBBPXY,1,1,0,0,0,1,0,0,1,...,0,NDP,HMO,HT,Bottom 45,Holdout,0.036019,0.444071,0.038429,0.481481
2,003RZGBBBPXY,1,1,0,0,0,1,0,0,1,...,0,remaining,PPO,HT,Bottom 45,Holdout,0.032898,0.448825,0.038218,0.480059
3,005RGQBBBPXZ,1,1,0,0,0,1,0,0,1,...,0,NDP,HMO,HT,None,None,0.032978,0.388598,0.048713,0.529710
4,00BGXGBBBPXY,1,1,0,0,0,1,0,0,1,...,0,remaining,PPO,HT,Bottom 45,Holdout,0.036019,0.444071,0.038429,0.481481


In [25]:

data1 = data[data['expe_group'] == 'TT']

data1['prob'] = data1['TT'].apply(lambda x : x)

data2 = data[data['expe_group'] == 'TH']
data2['prob'] = data2['T_WBC_H_MSV'].apply(lambda x : x)

data3 = data[data['expe_group'] == 'HT']
data3['prob'] = data3['H_WBC_T_MSV'].apply(lambda x : x)

data4 = data[data['expe_group'] == 'HH']
data4['prob'] = data4['HH'].apply(lambda x : x)
print(data1.proxy_id .count(), data2.proxy_id .count(), data3.proxy_id .count(), data4.proxy_id .count())

61027 5025 50065 4017


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [26]:

data.expe_group.value_counts()

TT    61027
HT    50065
TH     5025
HH     4017
Name: expe_group, dtype: int64

In [27]:
data4.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,sales_channel,product_segment,expe_group,decile_group,call_type,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT,prob
7,00Q1NBBBBPXZ,1,1,0,0,0,1,0,0,0,...,Strategic,HMO,HH,Bottom 45,Holdout,0.036019,0.444071,0.038429,0.481481,0.036019
34,03FF9KBBBPXY,1,1,0,0,0,1,0,0,0,...,NDP,HMO,HH,None,None,0.036525,0.406253,0.044414,0.512808,0.036525
53,05D61BBBBPXY,1,1,0,0,0,1,0,0,0,...,Strategic,HMO,HH,Top1,Holdout,0.027631,0.365612,0.042908,0.563849,0.027631
58,05GV1BBBBPXZ,1,1,0,0,0,1,0,0,0,...,remaining,PPO,HH,Bottom 45,Holdout,0.036019,0.444071,0.038429,0.481481,0.036019
80,06L1SQBBBPXY,1,1,0,0,0,1,0,0,0,...,remaining,PPO,HH,Bottom 45,Holdout,0.032898,0.448825,0.038218,0.480059,0.032898


In [28]:
new_data = pd.concat([data1, data2])
new_data = pd.concat([new_data, data3])
new_data = pd.concat([new_data, data4])

In [29]:

new_data.proxy_id.count()

120134

In [30]:
new_data.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,sales_channel,product_segment,expe_group,decile_group,call_type,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT,prob
6,00KXDBBBBPXY,1,1,0,0,1,0,0,0,1,...,NDP,PPO,TT,Bottom 45,Live,0.036019,0.444071,0.038429,0.481481,0.481481
8,00QSXKBBBPXY,1,1,0,0,1,0,0,0,1,...,NDP,PPO,TT,Mid23,IVR,0.032978,0.388598,0.048713,0.529710,0.529710
9,00VZBBBBBPXY,1,1,0,0,1,0,0,0,1,...,NDP,PPO,TT,Top1,Live,0.027631,0.365612,0.042908,0.563849,0.563849
11,010RSKBBBPXY,1,1,0,0,1,0,0,0,1,...,Strategic,HMO,TT,None,None,0.032978,0.388598,0.048713,0.529710,0.529710
12,0138GRBBBPXY,1,1,0,0,1,0,0,0,1,...,NDP,PPO,TT,Bottom 45,Live,0.032898,0.448825,0.038218,0.480059,0.480059


In [31]:
new_data['weight'] = new_data['prob'].apply(lambda x :1/x)

In [32]:
new_data.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,product_segment,expe_group,decile_group,call_type,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT,prob,weight
6,00KXDBBBBPXY,1,1,0,0,1,0,0,0,1,...,PPO,TT,Bottom 45,Live,0.036019,0.444071,0.038429,0.481481,0.481481,2.076923
8,00QSXKBBBPXY,1,1,0,0,1,0,0,0,1,...,PPO,TT,Mid23,IVR,0.032978,0.388598,0.048713,0.529710,0.529710,1.887824
9,00VZBBBBBPXY,1,1,0,0,1,0,0,0,1,...,PPO,TT,Top1,Live,0.027631,0.365612,0.042908,0.563849,0.563849,1.773525
11,010RSKBBBPXY,1,1,0,0,1,0,0,0,1,...,HMO,TT,None,None,0.032978,0.388598,0.048713,0.529710,0.529710,1.887824
12,0138GRBBBPXY,1,1,0,0,1,0,0,0,1,...,PPO,TT,Bottom 45,Live,0.032898,0.448825,0.038218,0.480059,0.480059,2.083078


In [33]:
new_data['std_col'] = 1

In [34]:
new_data['expe_group'].unique()

array(['TT', 'TH', 'HT', 'HH'], dtype=object)

In [35]:
for i in list(new_data['expe_group'].unique()):
    new_data['std_col'][new_data['expe_group'] == i ] = new_data['weight']/new_data['weight'][new_data['expe_group'] == i].mean()

/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
new_data.groupby('expe_group')['std_col'].mean()

expe_group
HH    1.0
HT    1.0
TH    1.0
TT    1.0
Name: std_col, dtype: float64

In [37]:
new_data.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,...,expe_group,decile_group,call_type,HH,H_WBC_T_MSV,T_WBC_H_MSV,TT,prob,weight,std_col
6,00KXDBBBBPXY,1,1,0,0,1,0,0,0,1,...,TT,Bottom 45,Live,0.036019,0.444071,0.038429,0.481481,0.481481,2.076923,1.055058
8,00QSXKBBBPXY,1,1,0,0,1,0,0,0,1,...,TT,Mid23,IVR,0.032978,0.388598,0.048713,0.529710,0.529710,1.887824,0.958998
9,00VZBBBBBPXY,1,1,0,0,1,0,0,0,1,...,TT,Top1,Live,0.027631,0.365612,0.042908,0.563849,0.563849,1.773525,0.900935
11,010RSKBBBPXY,1,1,0,0,1,0,0,0,1,...,TT,None,None,0.032978,0.388598,0.048713,0.529710,0.529710,1.887824,0.958998
12,0138GRBBBPXY,1,1,0,0,1,0,0,0,1,...,TT,Bottom 45,Live,0.032898,0.448825,0.038218,0.480059,0.480059,2.083078,1.058185


In [38]:
new_data.columns

Index(['proxy_id', 'wbc_ind', 'msv_ind', 'rtn_rnf_ind', 'return_pilot_ind',
       'wbc_t', 'wbc_h', 'rr_t', 'rr_h', 'msv_t', 'msv_h', 'return_pilot_t',
       'return_pilot_h', 'decile', 'lis_ind', 'vol_churn', 'sales_channel',
       'product_segment', 'expe_group', 'decile_group', 'call_type', 'HH',
       'H_WBC_T_MSV', 'T_WBC_H_MSV', 'TT', 'prob', 'weight', 'std_col'],
      dtype='object')

In [39]:
new_data['churn_after_bal'] = new_data['vol_churn']* new_data['std_col']

In [40]:
new_data.groupby('expe_group').vol_churn.mean()

expe_group
HH    0.047299
HT    0.037631
TH    0.048557
TT    0.044144
Name: vol_churn, dtype: float64

In [41]:
new_data.groupby('expe_group').churn_after_bal.mean()

expe_group
HH    0.048204
HT    0.038874
TH    0.047011
TT    0.042873
Name: churn_after_bal, dtype: float64

In [47]:
new_data.groupby('expe_group').proxy_id.count()

expe_group
HH     4017
HT    50065
TH     5025
TT    61027
Name: proxy_id, dtype: int64

In [42]:
a = pd.DataFrame(new_data.groupby(['expe_group', 'call_type']).churn_after_bal.mean())
a

churn_after_bal
expe_group call_type                 
HH         Holdout           0.044278
HT         Holdout           0.033402
TH         IVR               0.036636
           Live              0.036696
TT         IVR               0.034978
           Live              0.033275

In [43]:
a = pd.DataFrame(new_data.groupby(['expe_group', 'call_type']).proxy_id.count())
a

proxy_id
expe_group call_type          
HH         Holdout        3647
HT         Holdout       45815
TH         IVR            1244
           Live           2705
TT         IVR           15186
           Live          33761

In [44]:
from statsmodels.stats.proportion import proportions_ztest
import scipy
from scipy import stats
from scipy.stats import norm

import  numpy as np
def ztest_prop(df0, i ):
    d1 = df0[df0['expe_group']== i  ]
    d2 = df0[df0['expe_group']=='HH']
    count=np.array([d1['churn_after_bal'].sum(), d2['churn_after_bal'].sum()])
    nobs = np.array([d1['churn_after_bal'].count(), d2['churn_after_bal'].count()])
    stat, pval = proportions_ztest(count, nobs,alternative='smaller')   #alter : T<H
    pval1 = stats.norm.cdf(stat)
    print (pval , stat,count,nobs)
    if pval < 0.05:
        print('significant at 95%')
    elif pval>0.05 and pval <= 0.10 :
        print('significant at 90%')
    elif pval >0.10 and pval<= .20 :
        print('significant at 80%')
    else:
        print('Same Prop')

        
for i in ['HT', 'TH', 'TT'] : 
    print (i)
    ztest_prop(new_data, i)

HT
0.0017577576054899535 -2.918649266015953 [1946.20442146  193.63422299] [50065  4017]
significant at 95%
TH
0.39560399494647985 -0.26474229893265183 [236.23211518 193.63422299] [5025 4017]
Same Prop
TT
0.05372971973538477 -1.6097179729416262 [2616.39556739  193.63422299] [61027  4017]
significant at 90%


In [45]:
a = pd.DataFrame(new_data.groupby(['decile','expe_group']).churn_after_bal.mean())
a.reset_index(inplace=True)
a

,decile,expe_group,churn_after_bal
0,0,HH,0.000000
1,0,HT,0.221129
2,0,TH,0.000000
3,0,TT,0.099870
4,1,HH,0.094297
5,1,HT,0.077514
6,1,TH,0.102493
7,1,TT,0.072982
8,2,HH,0.061825
9,2,HT,0.054107


In [44]:
b= pd.DataFrame(new_data.groupby(['decile','expe_group']).churn_after_bal.sum())
b.reset_index(inplace=True)
b

,decile,expe_group,churn_after_bal
0,0,HH,0.000000
1,0,HT,6.190192
2,0,TH,0.000000
3,0,TT,6.892239
4,1,HH,50.831149
5,1,HT,552.700855
6,1,TH,85.823773
7,1,TT,802.876419
8,2,HH,40.561455
9,2,HT,418.150684


In [45]:
excelfilename = "test.xlsx" 
with pd.ExcelWriter(excelfilename) as writer: 
    a.to_excel(writer,sheet_name = 'dec' , startrow = 1 ) 
    b.to_excel(writer,sheet_name = 'lis' , startrow = 1 )

# FACTOR

In [56]:
sdf = hc.sql(""" select a.*, case when wbc_t = 1 and msv_t = 1 then 'TT' when wbc_t = 1 and msv_t = 0 then 'TH'
when wbc_t = 0 and msv_t = 1 then 'HT'
else 'HH' end as expe_group , decile_group from 
cust_exp_enc.a454533_oep_2021_renew_mbr_campaigns_v2 a
Left Join cust_exp_enc.n275675_Welcome_back_call_2021_master_v1 b
on a.proxy_id = b.proxy_id
where msv_ind = 1 and wbc_ind = 1 and rtn_rnf_ind = 0 and return_pilot_ind = 0
""")
data_frame = sdf.toPandas()
data_frame.head()

,proxy_id,wbc_ind,msv_ind,rtn_rnf_ind,return_pilot_ind,wbc_t,wbc_h,rr_t,rr_h,msv_t,msv_h,return_pilot_t,return_pilot_h,decile,lis_ind,vol_churn,sales_channel,product_segment,expe_group,decile_group
0,001LGBBBBPXY,1,1,0,0,0,1,0,0,1,0,0,0,5,0,0,Strategic,HMO,HT,Bottom 45
1,002317BBBPXY,1,1,0,0,0,1,0,0,1,0,0,0,4,0,0,NDP,HMO,HT,Bottom 45
2,003RZGBBBPXY,1,1,0,0,0,1,0,0,1,0,0,0,5,0,0,remaining,PPO,HT,Bottom 45
3,005RGQBBBPXZ,1,1,0,0,0,1,0,0,1,0,0,0,2,1,0,NDP,HMO,HT,None
4,00BGXGBBBPXY,1,1,0,0,0,1,0,0,1,0,0,0,4,0,0,remaining,PPO,HT,Bottom 45


In [57]:
from statsmodels.stats.proportion import proportions_ztest
import scipy
from scipy import stats
from scipy.stats import norm

import  numpy as np
def ztest_prop(df0, i ):
    d1 = df0[df0['expe_group']== i  ]
    d2 = df0[df0['expe_group']=='HH']
    count=np.array([d1['churn_after_bal'].sum(), d2['churn_after_bal'].sum()])
    nobs = np.array([d1['churn_after_bal'].count(), d2['churn_after_bal'].count()])
    stat, pval = proportions_ztest(count, nobs,alternative='smaller')   #alter : T<H
    pval1 = stats.norm.cdf(stat)
    return (pval ,i)

import statsmodels.api as sm

In [58]:
def factor(data_frame , factor ,fac_val):
    print(factor,fac_val)
    df = data_frame[data_frame[factor] == fac_val ]
    
    X = df[['decile']]
    catg_col = ['decile']
    X = pd.get_dummies(X, columns = catg_col, prefix = catg_col, drop_first = True)
    Y = df['expe_group']
    X_1=sm.add_constant(X)


    mnlogit_model=sm.MNLogit(Y,X_1)
    result1=mnlogit_model.fit(method='bfgs')


    a = pd.DataFrame(result1.predict(X_1)) 
    a.rename(columns = {0:'HH',1: 'H_WBC_T_MSV', 2:'T_WBC_H_MSV', 3 : 'TT'}, inplace= True)

    data = pd.concat([df,a], axis = 1 )

    data1 = data[data['expe_group'] == 'TT']

    data1['prob'] = data1['TT'].apply(lambda x : x)

    data2 = data[data['expe_group'] == 'TH']
    data2['prob'] = data2['T_WBC_H_MSV'].apply(lambda x : x)

    data3 = data[data['expe_group'] == 'HT']
    data3['prob'] = data3['H_WBC_T_MSV'].apply(lambda x : x)

    data4 = data[data['expe_group'] == 'HH']
    data4['prob'] = data4['HH'].apply(lambda x : x)
    #print(data1.mzb_indiv_id .count(), data2.mzb_indiv_id .count(), data3.mzb_indiv_id .count(), data4.mzb_indiv_id .count())

    new_data = pd.concat([data1, data2])
    new_data = pd.concat([new_data, data3])
    new_data = pd.concat([new_data, data4])

    new_data['weight'] = new_data['prob'].apply(lambda x :1/x)
    new_data['std_col'] = 1
    for i in list(new_data['expe_group'].unique()):
        new_data['std_col'][new_data['expe_group'] == i ] = new_data['weight']/new_data['weight'][new_data['expe_group'] == i].mean()

    new_data['churn_after_bal'] = new_data['vol_churn']* new_data['std_col']

    col = ['expe_group' , factor ,'vol_churn', 'churn_after_bal', 'proxy_id']
    sale = pd.DataFrame(columns = col)

    a = pd.DataFrame(new_data.groupby([factor,'expe_group']).churn_after_bal.mean())
    a.reset_index(inplace=True)
    b = pd.DataFrame(new_data.groupby([factor,'expe_group']).vol_churn.mean())
    b.reset_index(inplace=True)
    c = pd.merge(b,a,how='inner')
    d = pd.DataFrame(new_data.groupby([factor,'expe_group']).proxy_id.count())
    d.reset_index(inplace=True)
    d = pd.merge(d,c,how = 'inner')
    sale = pd.concat([sale,d], axis = 0)
 
    
    columns = ['Pvalue', 'expe_group']
    test1 = pd.DataFrame(columns=columns)
    for i in ['HT', 'TH', 'TT']:
        test2 = pd.DataFrame([ztest_prop(new_data,i)],columns=columns)
        test1= pd.concat([test1,test2],ignore_index=True) 
    
    final = pd.merge(sale,test1, on = 'expe_group' , how = 'left')
    return (final)


In [59]:
data_frame.decile_group.value_counts()

Bottom 45    59801
Mid23        31614
Top1         10943
Name: decile_group, dtype: int64

In [60]:
factor(data_frame,'decile_group','Top1')

decile_group Top1
         Current function value: 0.934397
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,decile_group,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,Top1,0.070461,0.070461,369,NaN
1,HT,Top1,0.052108,0.052055,5028,0.064378
2,TH,Top1,0.083815,0.083815,346,0.748472
3,TT,Top1,0.052500,0.052488,5200,0.069417


In [61]:
factor(data_frame,'decile_group','Mid23')

decile_group Mid23
Optimization terminated successfully.
         Current function value: 0.972010
         Iterations: 30
         Function evaluations: 31
         Gradient evaluations: 31


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,expe_group,decile_group,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,Mid23,0.053454,0.053580,1216,NaN
1,HT,Mid23,0.041158,0.041104,14019,0.018849
2,TH,Mid23,0.053625,0.053626,1324,0.502023
3,TT,Mid23,0.043507,0.043560,15055,0.051192


In [62]:
factor(data_frame,'decile_group','Bottom 45')

decile_group Bottom 45
Optimization terminated successfully.
         Current function value: 0.952757
         Iterations: 31
         Function evaluations: 32
         Gradient evaluations: 32


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,expe_group,decile_group,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,Bottom 45,0.033948,0.033621,2062,NaN
1,HT,Bottom 45,0.025030,0.025041,26768,0.008791
2,TH,Bottom 45,0.021062,0.021054,2279,0.005388
3,TT,Bottom 45,0.025408,0.025406,28692,0.011731


In [78]:
factor(data_frame,'lis_ind',1)

lis_ind 1
         Current function value: 0.857835
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,lis_ind,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,1,0.075521,0.076822,384,NaN
1,HT,1,0.088759,0.088821,4439,0.787217
2,TH,1,0.090659,0.092269,1092,0.820651
3,TT,1,0.086421,0.086559,12335,0.748343


In [79]:
factor(data_frame,'lis_ind',0)

lis_ind 0
         Current function value: 0.956904
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,lis_ind,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,0,0.044304,0.043630,3634,NaN
1,HT,0,0.032665,0.032608,45645,0.000187
2,TH,0,0.037075,0.037806,3938,0.099785
3,TT,0,0.033402,0.033477,48740,0.000578


In [81]:
factor(data_frame,'product_segment','HMO')

product_segment HMO
         Current function value: 1.065881
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,product_segment,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,HMO,0.054154,0.055537,1902,NaN
1,HT,HMO,0.053373,0.056690,12628,0.580461
2,TH,HMO,0.056129,0.052740,2423,0.343079
3,TT,HMO,0.057689,0.055390,16329,0.489438


In [82]:
factor(data_frame,'product_segment','PPO')

product_segment PPO
         Current function value: 0.898435
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,product_segment,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,PPO,0.041115,0.040616,2116,NaN
1,HT,PPO,0.032331,0.033283,37456,0.034439
2,TH,PPO,0.041811,0.040523,2607,0.493585
3,TT,PPO,0.039154,0.038165,44746,0.282933


In [83]:
factor(data_frame,'sales_channel','Strategic')


sales_channel Strategic
         Current function value: 0.966527
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,sales_channel,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,Strategic,0.071245,0.072876,1165,NaN
1,HT,Strategic,0.064129,0.065588,13239,0.168689
2,TH,Strategic,0.087291,0.085644,1558,0.887601
3,TT,Strategic,0.072874,0.071690,17098,0.439717


In [84]:
factor(data_frame,'sales_channel','NDP')



sales_channel NDP
         Current function value: 0.960289
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,sales_channel,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,NDP,0.036898,0.037010,1870,NaN
1,HT,NDP,0.028479,0.028881,22648,0.022932
2,TH,NDP,0.037869,0.039188,2271,0.642041
3,TT,NDP,0.033065,0.032472,26735,0.143288


In [85]:

factor(data_frame,'sales_channel','remaining')

sales_channel remaining
         Current function value: 0.925522
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/users/n323489/.conda/envs/n323489_pyconnect_py37/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,expe_group,sales_channel,vol_churn,churn_after_bal,proxy_id,Pvalue
0,HH,remaining,0.038657,0.038570,983,NaN
1,HT,remaining,0.027541,0.028293,14197,0.031557
2,TH,remaining,0.019151,0.018955,1201,0.002761
3,TT,remaining,0.032711,0.031657,17242,0.115602
